<a href="https://colab.research.google.com/github/frankl1/Introduction-to-Tensorflow-for-Deep-Learning/blob/master/exercise_flowers_with_data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the libraries

In [0]:
import numpy as np
import os
import glob
import shutil
import matplotlib.pyplot as plt

In [7]:
try:
  %tensorflow_version 2.x
except:
  pass

import tensorflow as tf
from tf.keras import layers as klayers
from tf.keras.models import Sequential 
from tf.keras.preprocessing.image import ImageDataGenerator

ModuleNotFoundError: ignored

# Loading the data

In [0]:
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file_path = tf.keras.utils.get_file(fname="flower_photos.tgz", origin=_URL, extract=True)

base_dir = os.path.join(os.path.dirname(zip_file_path), "flower_photos")

In [0]:
classes = ['roses', 'daisy', 'dandelion', 'sunflowers', 'tulips']